Zhang Chao, 2025.04.25<br>
Three components of ET changes

In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import regionmask
import matplotlib.pyplot as plt
import geopandas as gpd
import os
import sys
import scipy
from matplotlib.colors import Normalize
import cartopy.io.shapereader as shpreader
import matplotlib.colors as mcolors
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))
from utils import getSigFlag, stats_regionmean
sys.path.append('/home/climate/chaoz/code/utils/')
from plot_utils import plot_settings

In [2]:
os.chdir('/home/climate/chaoz/project/03Irr_Ts_CN/processed/')
Ir = xr.open_dataset('IrrMap_stable_CN.nc')
ET = xr.open_dataset('ET_PMLv2_Yrmean_1km_CN_2001_2020.nc').where(Ir.Ir==1)

shp_cn = gpd.read_file('../shapefile/ChinaAll.shp')
shp_nanhai = gpd.read_file('../shapefile/Nanhai.shp')
shp_climzone = gpd.read_file('../shapefile/ClimateZone_3.shp')

In [3]:
df_Ec = stats_regionmean(ET,'Ec',shp_climzone)
df_Ei = stats_regionmean(ET,'Ei',shp_climzone)
df_Es = stats_regionmean(ET,'Es',shp_climzone)

In [4]:
def plot_regionwater(ax,data,ylim,yticks,xlabelFlag,ytickflag,color,No):
    x = range(2001, 2021)
    y = data

    ###
    ax.plot(x, y, c=color)

    ax.set_ylabel('')
    ax.set_xlim(2000,2021)
    
    parameter = np.polyfit(x, y, 1)
    f = np.poly1d(parameter)
    r, sig = scipy.stats.pearsonr(x, y)
    ax.plot(x, f(x), ls='--', c=color)
    ax.text(-0.12,1.05, No, transform=ax.transAxes, c='k',
             weight='bold',fontsize=14)  # ,style='italic'
    
    ax.set_ylim(ylim)
    if xlabelFlag == True:
        ax.set_xticks([2005,2015], [2005,2015])
    else:
        ax.set_xticks([2005,2015], [])
        
    if ytickflag:
        ax.set_yticks(yticks,yticks)
    else:
        ax.set_yticks(yticks,[])
        
    ax.text(0.10, 0.90, "slope = %.2f$^{%s}$" %
             (parameter[0] * 10, getSigFlag(sig)),
             transform=ax.transAxes, c='k',fontsize=12)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    return ax

In [ ]:
fig, axes = plt.subplots(nrows=3,ncols=4,figsize=(9,5),sharex=True)
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plot_settings()
x = range(0,df_Ec.shape[0])
colors = ['#4EAB90','#457B9D','#834026']
d=365
yticks1=[100,200,300,400,500]
yticks2=[0,10,20,30,40]
yticks3=[100,150,200,250,300]
ylim1,ylim2,ylim3 = [100,510],[0,45],[80,310]
colors = ['#4EAB90','#457B9D','#834026']

plot_regionwater(axes[0,0],df_Ec.iloc[:,0+1]*d,ylim1,yticks1,False,True,colors[0],'a')
plot_regionwater(axes[0,1],df_Ec.iloc[:,1+1]*d,ylim1,yticks1,False,False,colors[0],'b')
plot_regionwater(axes[0,2],df_Ec.iloc[:,2+1]*d,ylim1,yticks1,False,False,colors[0],'c')
plot_regionwater(axes[0,3],df_Ec.iloc[:,3+1]*d,ylim1,yticks1,False,False,colors[0],'d')

plot_regionwater(axes[1,0],df_Ei.iloc[:,0+1]*d,ylim2,yticks2,False,True,colors[1],'e')
plot_regionwater(axes[1,1],df_Ei.iloc[:,1+1]*d,ylim2,yticks2,False,False,colors[1],'f')
plot_regionwater(axes[1,2],df_Ei.iloc[:,2+1]*d,ylim2,yticks2,False,False,colors[1],'g')
plot_regionwater(axes[1,3],df_Ei.iloc[:,3+1]*d,ylim2,yticks2,False,False,colors[1],'h')

plot_regionwater(axes[2,0],df_Es.iloc[:,0+1]*d,ylim3,yticks3,True,True,colors[2],'i')
plot_regionwater(axes[2,1],df_Es.iloc[:,1+1]*d,ylim3,yticks3,True,False,colors[2],'j')
plot_regionwater(axes[2,2],df_Es.iloc[:,2+1]*d,ylim3,yticks3,True,False,colors[2],'k')
plot_regionwater(axes[2,3],df_Es.iloc[:,3+1]*d,ylim3,yticks3,True,False,colors[2],'l')

axes[0,0].set_ylabel('Ec [mm]')
axes[1,0].set_ylabel('Ei [mm]')
axes[2,0].set_ylabel('Es [mm]')

axes[0,0].set_title('China')
axes[0,1].set_title('Arid')
axes[0,2].set_title('Semi arid/humid')
axes[0,3].set_title('Humid')

plt.tight_layout()
plt.savefig('../figures/Figure_S06.png',dpi=300)